<a href="https://colab.research.google.com/github/shweta7122/Sentiment-Analysis/blob/main/App_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit

In [2]:
import streamlit as st
import warnings
warnings.filterwarnings("ignore")
# EDA Pkgs
import pandas as pd
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler
import re
import textblob
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import openpyxl
import time
import tqdm

In [98]:
#To Hide Warnings
st.set_option('deprecation.showfileUploaderEncoding', False)
st.set_option('deprecation.showPyplotGlobalUse', False)
# Viz Pkgs
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
import seaborn as sns
#sns.set_style('darkgrid')
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [99]:
STYLE = """
<style>
img {
    max-width: 100%;
}
</style> """


In [1]:
def main():
    """ Common ML Dataset Explorer """
    #st.title("Live twitter Sentiment analysis")
    #st.subheader("Select a Twitter Handle for which you'd like to get the sentiment analysis:")

    html_temp = """
	<div style="background-color:tomato;"><p style="color:white;font-size:25px;padding:9px">Live twitter Sentiment analysis</p></div>
	"""
    st.markdown(html_temp, unsafe_allow_html=True)
    st.subheader("Select a Twitter Handle which you'd like to get the sentiment analysis on :")


In [76]:
################# Twitter API Connection #######################
consumerKey = "kDgO0qZRdVkKICung7MkPl2QU"
consumerSecret = "ZOQhlwSEMhZUtdVPjc9CG4kaGCEWdR6AT9e2SJiJXoIMHxnIDi"
accessToken = "1445776538599768074-Vl3Udixl0J4EwIokYsboiQYbkviqin"
accessTokenSecret = "9xBgvVedGyrtUsofSvVLqM9i499AfSoapl0XtBzaVamrL"

In [100]:
# Creating the authentication data
authenticate = tweepy.OAuthHandler(consumerKey, consumerSecret)

# Set the access token and access token secret
authenticate.set_access_token(accessToken, accessTokenSecret)

# Create the API object
api = tweepy.API(authenticate, wait_on_rate_limit=True)

In [102]:
TwitterHandle = "JustinTrudeau"
posts = api.user_timeline(screen_name = TwitterHandle, count = 100, lang ='en', tweet_mode="extended")
df = pd.DataFrame([tweet.full_text for tweet in posts], columns=['Tweets'])

In [79]:
# Create a function to clean the texts
def cleanTxt(text):
  text = re.sub(r'@[A-Za-z0-9]+', '', text) #Removed @mentions
  text = re.sub(r'#', '', text) #Removed Hastags
  text = re.sub(r'RT[\s]+', '', text) #Removed RT
  text = re.sub(r'https?:\/\/\S+', '', text) # Removed hyperlinks

  return text

In [81]:
# Create a function to get the functionality
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

# Create a function to get the polarity
def getPolarity(text):
  return TextBlob(text).sentiment.polarity

# Create two new columns
df['Subjectivity'] = df['Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Tweets'].apply(getPolarity)


In [82]:
# Create a function to compute the sentiment analysis
def getAnalysis(score):
  if score < 0:
    return 'Negative'
  elif score == 0:
    return 'Neutral'
  else:
    return 'Positive'

df['Analysis'] = df['Polarity'].apply(getAnalysis)


In [83]:
from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

In [84]:
allWords = ' '.join( [twts for twts in df['Tweets']])
wordcloud = WordCloud(background_color='white', width=500, height=300, stopwords=stopwords,max_words=50, max_font_size=120, random_state=21).generate(allWords)

print(wordcloud)
fig = plt.figure(1, figsize=(15,12))
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.show()
fig.savefig("word1.png", dpi=100)

In [87]:
from PIL import Image
image = Image.open('logo.jpg')
st.image(image, caption='Twitter for Analytics',use_column_width=True)

2021-10-07 19:15:04.455 
  command:

    streamlit run /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [94]:
# Collect Input from user :
TwitterHandle = str()
TwitterHandle = str(st.text_input("Enter the TwitterHandle you are interested in (Press Enter once done)"))     
    
if len(TwitterHandle) > 0 :
        # Call the function to extract the data. pass the Twitter Handle and filename you want the data to be stored in.
        with st.spinner("Please wait, Tweets are being extracted"):
            get_tweets(TwitterHandle , Count=200)
        st.success('Tweets have been Extracted !!!!')    
           
    
        # Call function to get Clean tweets
        df['CleanTxt'] = df['Tweets'].apply(lambda x : cleanTxt(x))
    
        # Call function to get the Sentiments
        df["Polarity"] = df["Tweet"].apply(lambda x : getAnalysis(x))
        
        
        # Write Summary of the Tweets
        st.write("Total Tweets Extracted for the User '{}' are : {}".format(TwitterHandle,len(df.Tweets)))
        st.write("Total Positive Tweets are : {}".format(len(df[df["Analysis"]=="Positive"])))
        st.write("Total Negative Tweets are : {}".format(len(df[df["Analysis"]=="Negative"])))
        st.write("Total Neutral Tweets are : {}".format(len(df[df["Analysis"]=="Neutral"])))
        
        # See the Extracted Data : 
        if st.button("See the Extracted Data"):
            #st.markdown(html_temp, unsafe_allow_html=True)
            st.success("Below is the Extracted Data :")
            st.write(df.head(50))
        
        
        # get the countPlot
        if st.button("Get Count Plot for Different Sentiments"):
            st.success("Generating A Count Plot")
            st.subheader(" Count Plot for Different Sentiments")
            st.write(sns.countplot(df["Analysis"]))
            st.pyplot()
        
        # Piechart 
        if st.button("Get Pie Chart for Different Sentiments"):
            st.success("Generating A Pie Chart")
            a=len(df[df["Analysis"]=="Positive"])
            b=len(df[df["Analysis"]=="Negative"])
            c=len(df[df["Analysis"]=="Neutral"])
            d=np.array([a,b,c])
            explode = (0.1, 0.0, 0.1)
            st.write(plt.pie(d,shadow=True,explode=explode,labels=["Positive","Negative","Neutral"],autopct='%1.2f%%'))
            st.pyplot()
            
       
        
        ## Points to add 1. Make Backgroud Clear for Wordcloud 2. Remove keywords from Wordcloud
        
        
        # Create a Worlcloud
        if st.button("Get WordCloud for{}".format(TwitterHandle)):
            st.success("Generating A WordCloud for all tweets by {}".format(TwitterHandle))
            text = " ".join(review for review in df.cleanTxt)
            stopwords = set(STOPWORDS)
            text_newALL = prepCloud(text,TwitterHandle)
            wordcloud = WordCloud(stopwords=stopwords,max_words=800,max_font_size=70).generate(text_newALL)
            st.write(plt.imshow(wordcloud, interpolation='bilinear'))
            st.pyplot()
        
        
        #Wordcloud for Positive tweets only
        if st.button("Get WordCloud for all Positive Tweets by {}".format(TwitterHandle)):
            st.success("Generating A WordCloud for all Positive Tweets by {}".format(TwitterHandle))
            text_positive = " ".join(review for review in df[df["Analysis"]=="Positive"].cleanTxt)
            stopwords = set(STOPWORDS)
            text_new_positive = prepCloud(text_positive,TwitterHandle)
            #text_positive=" ".join([word for word in text_positive.split() if word not in stopwords])
            wordcloud = WordCloud(stopwords=stopwords,max_words=800,max_font_size=70).generate(text_new_positive)
            st.write(plt.imshow(wordcloud, interpolation='bilinear'))
            st.pyplot()
        
        
        #Wordcloud for Negative tweets only       
        if st.button("Get WordCloud for all Negative Tweets by {}".format(TwitterHandle)):
            st.success("Generating A WordCloud for all Positive Tweets by {}".format(TwitterHandle))
            text_negative = " ".join(review for review in df[df["Analysis"]=="Negative"].cleanTxt)
            stopwords = set(STOPWORDS)
            text_new_negative = prepCloud(text_negative,TwitterHandle)
            #text_negative=" ".join([word for word in text_negative.split() if word not in stopwords])
            wordcloud = WordCloud(stopwords=stopwords,max_words=800,max_font_size=70).generate(text_new_negative)
            st.write(plt.imshow(wordcloud, interpolation='bilinear'))
            st.pyplot()
        


In [95]:
    st.sidebar.header("About App")
    st.sidebar.info("A Twitter Sentiment analysis Project -CSDA1040")
    st.sidebar.text("Built with Streamlit")

    #st.sidebar.subheader("Scatter-plot setup")
    #box1 = st.sidebar.selectbox(label= "X axis", options = numeric_columns)
    #box2 = st.sidebar.selectbox(label="Y axis", options=numeric_columns)
    #sns.jointplot(x=box1, y= box2, data=df, kind = "reg", color= "red")
    #st.pyplot()



    if st.button("Exit"):
        st.balloons()

In [96]:
if __name__ == '__main__':
    main()